In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [4]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
edges

array([[75582903194484819, 75582834475009044],
       [75582903194484867, 75582834475009018],
       [75582903194484873, 75582834475009193],
       [75582903194484912, 75512534450307165],
       [75582903194484963, 75582834475009168],
       [75582903194485200, 75512534450307435],
       [75582903194485330, 75512534450307707],
       [75582903194485481, 75512534450307707],
       [75582903194485506, 75582971913961513],
       [75582903194485507, 75582971913961542],
       [75582903194485544, 75582971913961501],
       [75582903194485666, 75582971913961503],
       [75582903194485697, 75582971913961503],
       [75582903194485720, 75512534450307957],
       [75582903194485720, 75582971913961693],
       [75582903194485753, 75582971913961503]], dtype=uint64)

In [6]:
# cg.get_parents(np.unique(edges))
cg.get_parents([75582903194484819, 75582903194485753])
cg.get_parents([147640497232412675, 147640497232412673])
# cg.get_parents([147640497232412673, 147710934696067076, 147640497232412683,
#  147640565951889409])

array([291635563662934657, 219704963218014230], dtype=uint64)

In [7]:
print(cg.get_chunk_layer(219704963218014230))
print(cg.get_children(219704963218014230))
# print(cg.get_chunk_coordinates(223237144322244610))

3
[147640497232412673 147710934696067076 147640497232412683
 147640565951889409]


In [3]:
print(cg.get_chunk_id(223237144322244626))
print(cg.get_chunk_id(219704963218014230))
print(cg.get_parent_chunk_id(147640497232412673, 3))
print(cg.get_parent_chunk_id(147640497232412683, 3))

223237144322244608
219704963218014208
223237144322244608
223237144322244608


In [3]:
from pychunkedgraph.graph.edits_v2 import add_edge_v2
add_edge_v2(cg, edge=[75582903194484819, 75582903194485753])

bounding_chunk_ids [151165806388969472 151165806456078336 151165875108446208
 151165875175555072 151236175133147136 151236175200256000
 151236243852623872 151236243919732736]
node_id, children 223237144322244630 [147640497232412715 147640497232412673 147640497232412683]

children_chunk_ids [147640497232412672 147640497232412672 147640497232412672]


ValueError: Coordinate is out of range             layer: 12 bits/dim 1.             [50, 100, 0]; max = 2.